In [8]:
import pandas as pd
import matplotlib as plt
import numpy as np
from config import api_key
import json
import requests
from pathlib import Path
from sqlalchemy import create_engine, MetaData, Table, inspect, select
from sqlalchemy.orm import sessionmaker, Session

In [9]:
API_HOST = 'https://api.yelp.com/'
SEARCH_PATH = 'v3/businesses/search'
url = f"{API_HOST}{SEARCH_PATH}"

print(url)

https://api.yelp.com/v3/businesses/search


In [16]:
SEARCH_LIMIT = 50
SEARCH_OFFSET = 942
total_results = 1000

In [11]:
term = "Restaurants" 
location = "Perth WA"
limit = 50

In [12]:
#This cell is pretty much putting the url params into dictionaries so we can save it as a variable as suppose to writing everthing down when we do  
# ".get()"
url_params = {'term': term.replace(' ','+'),
              'location': location.replace(' ','+'),
              'limit': limit,
              'offset': SEARCH_OFFSET}

print(url_params)

{'term': 'Restaurants', 'location': 'Perth+WA', 'limit': 50, 'offset': 0}


In [13]:
# Cell 2: Define helper functions to manage offsets and data saving
def get_last_offset(file_path='last_offset.txt'):
    try:
        with open(file_path, 'r') as file:
            return int(file.read())
    except FileNotFoundError:
        return 0

def update_last_offset(offset, file_path='last_offset.txt'):
    with open(file_path, 'w') as file:
        file.write(str(offset))


In [17]:
last_offset = get_last_offset()
new_start_offset = last_offset + total_results  # Prepare to update for next run
all_data = []

for offset in range(SEARCH_OFFSET, total_results, SEARCH_LIMIT):
    url_params = {
        'term': 'Restaurants'.replace(' ', '+'),
        'location': 'Perth WA'.replace(' ', '+'),
        'limit': SEARCH_LIMIT,
        'offset': offset
    }
    
    url = f'{API_HOST}{SEARCH_PATH}'
    headers = {'Authorization': f'Bearer {api_key}'}
    
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()
    print(data)  # Print the entire JSON response for debugging
    # print(data.keys())
    
    if 'businesses' in data:
        all_data.append(pd.json_normalize(data["businesses"]))
    else:
        print("Error: 'businesses' key not found in API response.")
    
    
    # all_data.append(pd.json_normalize(data.get("businesses", [])))
print(all_data)

{'businesses': [], 'total': 942, 'region': {'center': {'longitude': 115.85932731628418, 'latitude': -31.948957166576726}}}
{'error': {'code': 'VALIDATION_ERROR', 'description': 'Too many results requested, limit+offset must be <= 1000.'}}
Error: 'businesses' key not found in API response.
[Empty DataFrame
Columns: []
Index: []]


In [15]:
# Process the fetched data as needed
combined_data = pd.concat(all_data, ignore_index=True)

file_name = 'yelp_data.csv'
file_path = Path(file_name)

# Check if the Excel file exists and append or write new data
if file_path.is_file():
    existing_data = pd.read_csv(file_path)
    updated_data = pd.concat([existing_data, combined_data], ignore_index=True)
else:
    updated_data = combined_data

# Export the updated data to Excel
updated_data.to_csv(file_name, index=False)

# Update the last offset used for the next run
update_last_offset(new_start_offset)

print(f"Data exported to {file_name}. Total businesses fetched: {len(combined_data)}.")


Data exported to yelp_data.csv. Total businesses fetched: 942.


In [ ]:
import sqlite3

# Assuming this function exists
def get_last_offset():
    # Logic to fetch last_offset from somewhere
    return 0  # Returning a default value for now

# Assuming this variable exists
json_data = {}  # Simulated data

last_offset = get_last_offset()
if last_offset is not None:
    new_start_offset = last_offset + total_results  # Prepare to update for next run
    all_data = []

    for offset in range(last_offset, last_offset + total_results, 50):
        data = json_data  # Simulating API response for the sake of example
        all_data.append(data)

    # Create SQLite database connection
    conn = sqlite3.connect('yelp_data.db')
    cur = conn.cursor()

    try:
        # Create table if not exists
        cur.execute('''CREATE TABLE IF NOT EXISTS yelp_data (
                       id TEXT PRIMARY KEY,
                       alias TEXT,
                       name TEXT,
                       image_url TEXT,
                       is_closed TEXT,
                       url TEXT,
                       review_count INTEGER,
                       categories TEXT,
                       rating REAL,
                       transactions TEXT,
                       price VARCHAR,
                       phone TEXT,
                       display_phone TEXT,
                       distance REAL,
                       latitude REAL,
                       longitude REAL,
                       address1 TEXT,
                       address2 TEXT,
                       address3 TEXT,
                       city TEXT,
                       zip_code TEXT,
                       country TEXT,
                       state TEXT,
                       display_address TEXT
                       )''')

        # Insert data into the database
        for data in all_data:
            for business in data.get('businesses', []):
                cur.execute('''INSERT OR REPLACE INTO yelp_data 
                               (id, alias, name, image_url, is_closed, url, review_count, categories, rating, 
                               transactions, price, phone, display_phone, distance, latitude, longitude, 
                               address1, address2, address3, city, zip_code, country, state, display_address) 
                               VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                            (business['id'], business['alias'], business['name'], business['image_url'], 
                             str(business['is_closed']), business['url'], business['review_count'], 
                             str(business['categories']), business['rating'], str(business['transactions']), 
                             business['price'], business['phone'], business['display_phone'], 
                             business.get('distance', None), business['coordinates']['latitude'], 
                             business['coordinates']['longitude'], business['location']['address1'], 
                             business['location'].get('address2', None), business['location'].get('address3', None), 
                             business['location']['city'], business['location']['zip_code'], 
                             business['location']['country'], business['location']['state'], 
                             str(business['location']['display_address'])
                            ))

        # Commit changes
        conn.commit()
        print("Data exported to SQLite database.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        # Close the cursor and connection
        cur.close()
        conn.close()
else:
    print("Failed to retrieve last offset. Aborting.")
